In [8]:
#!pip install unidecode
#!python -m spacy download en_core_web_lg

import nltk
import re
import numpy as np
import pandas as pd
import unidecode
import spacy

nltk.download('punkt') # one time execution
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktLanguageVars
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en import English


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ranja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ranja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
class BulletPointLangVars(PunktLanguageVars):
    sent_end_chars = ('.', ';', ':')

class AnswerRetriever():
    
    stop_words = set(stopwords.words('english'))
    TAG_RE = re.compile(r'<[^>]+>')
            
    def preprocess(self, text):
        #convert to lower case
        text = text.lower()
        
        #remove HTML tags (used for data taken from websites directly)
        #text = TAG_RE.sub('', text)
        
        #remove accents (in case google translate glitches)
        text = unidecode.unidecode(text)
        
        #remove punctuation (easier for posTag)
        text = re.sub(r'[^\w\s]', '', text)
        
        #remove stopwords
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in self.stop_words]
        return (' '.join(str(w) for w in filtered_sentence))
  
    def cosine_similarity_calc(self, vec_1,vec_2):
        sim = np.dot(vec_1,vec_2)/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))
        return sim     

    def process(self, articles):
        processed_article = []
        prefix = " "
        for subarticle in articles: 
            for subsubarticle in subarticle:  
                prefix = subarticle[0]           
                if not (prefix==subsubarticle):
                    together = prefix+subsubarticle
                    processed_article.append(together)
        #display(processed_article)
        return processed_article
    

    
    def extractAnswer(self):
        #tokenizer = PunktSentenceTokenizer(lang_vars = BulletPointLangVars())
        subarticles = re.split('\n[0-9]+\.', self.keyArticle)
        subsubarticles = []
        for subarticle in subarticles:  
            subsubarticles.append(re.split('\n[a-z]\.', subarticle))
        processed_articles = self.process(subsubarticles)
        ar_df = pd.DataFrame(processed_articles, columns=['sentences'])
        ar_df['sentences'].apply(self.preprocess)
        ar_df['question'] = self.preprocess(self.question)
        embeddings = spacy.load('en_core_web_lg')
        ar_df['similarity'] = ar_df.apply(lambda x: self.cosine_similarity_calc(embeddings(x['sentences']).vector, embeddings(x['question']).vector), axis=1)       
        ar_df = ar_df.sort_values(['similarity'], ascending=False)
        return ar_df
                                                      
    def __init__(self, question, keyArticle):
        self.question = question
        self.keyArticle = keyArticle
        print(self.question)
        print(self.keyArticle)
    
        

In [15]:
question = "Can Obvion sell the collateral as a whole?"

keyArticle = '''1 If Obvion proceeds to the forced sale of the collateral, Obvion has the choice
between public sale (via a notary to be designated by it) and private sale
with the approval of the preliminary relief judge of the court.
2. In the event of a forced sale, Obvion is authorized:
a. to subdivide and/or subdivide the collateral into apartment rights;
b. to sell the collateral in parts;
c. sell multiple collateral together;
d. establish auction or sale conditions;
e. to establish easements and/or qualitative obligations;
f. to exercise all rights under the purchase agreement, insofar as the law opposes this
not resist;
g. to postpone or cancel a sale.
3. The mortgagor is obliged:
a. all desired information, data, documents and evidence about the collateral
to be handed over to the notary in charge of the sale. Or to the
preliminary relief judge of the court to whom a request for approval of a
private sale has been submitted.
b. to enable interested parties to view the collateral at a location to be determined by Obvion
set days and times;
c. in the case of public or private sale, the collateral before the commencement of
the auction (public sale) or before the date of the notarial deed of transfer
(private sale) completely and at the free disposal of the buyer
set. The costs of this will be borne by the mortgagor.
4. If Obvion also has a right of pledge on movable property that is intended to hold the collateral
on which the mortgage right is established, Obvion may use the pledged
sell the goods and the collateral together in accordance with the rules that apply to the law of
mortgage.
5. Obvion reduces the debts secured by the right of mortgage with the
proceeds of the forced sale. Obvion is authorized to determine which debts are
the yield are reduced and also the order in which and the extent and manner in which
it happens.
'''

ar = AnswerRetriever(question, keyArticle)
answer_df = ar.extractAnswer()
display(answer_df)


Can Obvion sell the collateral as a whole?
1 If Obvion proceeds to the forced sale of the collateral, Obvion has the choice
between public sale (via a notary to be designated by it) and private sale
with the approval of the preliminary relief judge of the court.
2. In the event of a forced sale, Obvion is authorized:
a. to subdivide and/or subdivide the collateral into apartment rights;
b. to sell the collateral in parts;
c. sell multiple collateral together;
d. establish auction or sale conditions;
e. to establish easements and/or qualitative obligations;
f. to exercise all rights under the purchase agreement, insofar as the law opposes this
not resist;
g. to postpone or cancel a sale.
3. The mortgagor is obliged:
a. all desired information, data, documents and evidence about the collateral
to be handed over to the notary in charge of the sale. Or to the
preliminary relief judge of the court to whom a request for approval of a
private sale has been submitted.
b. to enable interested p

['1 If Obvion proceeds to the forced sale of the collateral, Obvion has the choice\nbetween public sale (via a notary to be designated by it) and private sale\nwith the approval of the preliminary relief judge of the court.',
 ' In the event of a forced sale, Obvion is authorized:\na. to subdivide and/or subdivide the collateral into apartment rights;\nb. to sell the collateral in parts;\nc. sell multiple collateral together;\nd. establish auction or sale conditions;\ne. to establish easements and/or qualitative obligations;\nf. to exercise all rights under the purchase agreement, insofar as the law opposes this\nnot resist;\ng. to postpone or cancel a sale.',
 ' The mortgagor is obliged:\na. all desired information, data, documents and evidence about the collateral\nto be handed over to the notary in charge of the sale. Or to the\npreliminary relief judge of the court to whom a request for approval of a\nprivate sale has been submitted.\nb. to enable interested parties to view the col

,sentences,question,similarity
2,"In the event of a forced sale, Obvion is auth...",obvion sell collateral whole,0.759464
1,"In the event of a forced sale, Obvion is auth...",obvion sell collateral whole,0.740597
0,"In the event of a forced sale, Obvion is auth...",obvion sell collateral whole,0.732394
9,The mortgagor is obliged: in the case of publ...,obvion sell collateral whole,0.710577
7,The mortgagor is obliged: all desired informa...,obvion sell collateral whole,0.699711
3,"In the event of a forced sale, Obvion is auth...",obvion sell collateral whole,0.684860
4,"In the event of a forced sale, Obvion is auth...",obvion sell collateral whole,0.683273
5,"In the event of a forced sale, Obvion is auth...",obvion sell collateral whole,0.680283
6,"In the event of a forced sale, Obvion is auth...",obvion sell collateral whole,0.680112
8,The mortgagor is obliged: to enable intereste...,obvion sell collateral whole,0.663402
